# 2-2,自动微分机制

神经网络通常依赖反向传播求梯度来更新网络参数，求梯度过程通常是一件非常复杂而容易出错的事情。

而深度学习框架可以帮助我们自动地完成这种求梯度运算。

Pytorch一般通过反向传播 backward 方法 实现这种求梯度计算。该方法求得的梯度将存在对应自变量张量的grad属性下。

除此之外，也能够调用torch.autograd.grad 函数来实现求梯度计算。

这就是Pytorch的自动微分机制。

In [ ]:
import torch

print("torch.__version__=" + torch.__version__)

### 一，利用backward方法求导数

backward 方法通常在一个标量张量上调用，该方法求得的梯度将存在对应自变量张量的grad属性下。

如果调用的张量非标量，则要传入一个和它同形状 的gradient参数张量。

相当于用该gradient参数张量与调用张量作向量点乘，得到的标量结果再反向传播。


**1, 标量的反向传播**

In [ ]:
import torch  # 导入PyTorch库

# 定义函数 f(x) = a*x**2 + b*x + c 的参数
x = torch.tensor(0.0, requires_grad=True)  # 创建一个张量 x，初始值为 0.0，需要计算梯度
a = torch.tensor(1.0)  # 定义参数 a，这里取值为 1.0
b = torch.tensor(-2.0)  # 定义参数 b，这里取值为 -2.0
c = torch.tensor(1.0)  # 定义参数 c，这里取值为 1.0

# 计算函数值 f(x) = a*x**2 + b*x + c
y = a * torch.pow(x, 2) + b * x + c

# 使用反向传播计算导数
y.backward()  # 使用backward()方法计算 y 对 x 的导数

# 提取导数值
dy_dx = x.grad  # x.grad 包含了在 x = 0 处的导数值

# 打印导数值
print(dy_dx)  # 打印导数值，即 f'(0) 的值


**2, 非标量的反向传播**

In [ ]:
import torch  # 导入PyTorch库

# 定义函数 f(x) = a*x**2 + b*x + c 的参数
x = torch.tensor([[0.0, 0.0], [1.0, 2.0]], requires_grad=True)  # 创建一个多维张量 x，初始值为二维数组，需要计算梯度
a = torch.tensor(1.0)  # 定义参数 a，这里取值为 1.0
b = torch.tensor(-2.0)  # 定义参数 b，这里取值为 -2.0
c = torch.tensor(1.0)  # 定义参数 c，这里取值为 1.0

# 计算函数值 f(x) = a*x**2 + b*x + c
y = a * torch.pow(x, 2) + b * x + c

# 定义梯度（导数）的权重，这里使用与 y 同维度的张量来指定每个分量的权重
gradient = torch.tensor([[1.0, 1.0], [1.0, 1.0]])

# 打印输入 x 和计算得到的输出 y
print("x:\n", x)
print("y:\n", y)

# 使用反向传播计算导数，并且使用指定的梯度权重
y.backward(gradient=gradient)

# 提取 x 关于 y 的梯度值
x_grad = x.grad

# 打印计算得到的梯度值
print("x_grad:\n", x_grad)

**3, 非标量的反向传播可以用标量的反向传播实现**

In [ ]:
import torch  # 导入PyTorch库

# 定义函数 f(x) = a*x**2 + b*x + c 的参数
x = torch.tensor([[0.0, 0.0], [1.0, 2.0]], requires_grad=True)  # 创建一个多维张量 x，初始值为二维数组，需要计算梯度
a = torch.tensor(1.0)  # 定义参数 a，这里取值为 1.0
b = torch.tensor(-2.0)  # 定义参数 b，这里取值为 -2.0
c = torch.tensor(1.0)  # 定义参数 c，这里取值为 1.0

# 计算函数值 f(x) = a*x**2 + b*x + c
y = a * torch.pow(x, 2) + b * x + c

# 定义梯度（导数）的权重，这里使用与 y 同维度的张量来指定每个分量的权重
gradient = torch.tensor([[1.0, 1.0], [1.0, 1.0]])

# 计算标量 z = Σ(y * gradient)
z = torch.sum(y * gradient)

# 打印输入 x 和计算得到的输出 y
print("x:", x)
print("y:", y)

# 使用反向传播计算导数
z.backward()

# 提取 x 关于 z 的梯度值
x_grad = x.grad

# 打印计算得到的梯度值
print("x_grad:\n", x_grad)


### 二，利用autograd.grad方法求导数

In [ ]:
import torch  # 导入PyTorch库

# 定义函数 f(x) = a*x**2 + b*x + c 的参数
x = torch.tensor(0.0, requires_grad=True)  # 创建一个张量 x，初始值为 0.0，需要计算梯度
a = torch.tensor(1.0)  # 定义参数 a，这里取值为 1.0
b = torch.tensor(-2.0)  # 定义参数 b，这里取值为 -2.0
c = torch.tensor(1.0)  # 定义参数 c，这里取值为 1.0

# 计算函数值 f(x) = a*x**2 + b*x + c
y = a * torch.pow(x, 2) + b * x + c

# 使用 torch.autograd.grad 计算一阶导数，并设置 create_graph=True 以允许创建更高阶的导数
dy_dx = torch.autograd.grad(y, x, create_graph=True)[0]

# 打印一阶导数
print("一阶导数 (dy/dx) 在 x=0 处的值：", dy_dx.data)

# 使用 torch.autograd.grad 计算二阶导数
dy2_dx2 = torch.autograd.grad(dy_dx, x)[0]

# 打印二阶导数
print("二阶导数 (d²y/dx²) 在 x=0 处的值：", dy2_dx2.data)


In [ ]:
import torch  # 导入PyTorch库

# 定义两个自变量 x1 和 x2，需要计算它们的梯度
x1 = torch.tensor(1.0, requires_grad=True)  # 创建一个张量 x1，初始值为 1.0，需要计算梯度
x2 = torch.tensor(2.0, requires_grad=True)  # 创建一个张量 x2，初始值为 2.0，需要计算梯度

# 定义两个因变量 y1 和 y2
y1 = x1 * x2
y2 = x1 + x2

# 使用 torch.autograd.grad 计算多个因变量关于多个自变量的导数

# 计算 y1 对 x1 和 x2 的导数
(dy1_dx1, dy1_dx2) = torch.autograd.grad(outputs=y1, inputs=[x1, x2], retain_graph=True)
print("y1 对 x1 的导数:", dy1_dx1.item())
print("y1 对 x2 的导数:", dy1_dx2.item())

# 计算 y1 和 y2 对 x1 和 x2 的导数，并且将它们的梯度结果相加
(dy12_dx1, dy12_dx2) = torch.autograd.grad(outputs=[y1, y2], inputs=[x1, x2])
print("y1 和 y2 对 x1 的导数:", dy12_dx1)
print("y1 和 y2 对 x2 的导数:", dy12_dx2)


### 三，利用自动微分和优化器求最小值

In [ ]:
import torch  # 导入PyTorch库

# 定义函数 f(x) = a*x**2 + b*x + c 的参数
x = torch.tensor(0.0, requires_grad=True)  # 创建一个张量 x，初始值为 0.0，需要计算梯度
a = torch.tensor(1.0)  # 定义参数 a，这里取值为 1.0
b = torch.tensor(-2.0)  # 定义参数 b，这里取值为 -2.0
c = torch.tensor(1.0)  # 定义参数 c，这里取值为 1.0

# 创建一个优化器，使用随机梯度下降（SGD）来优化变量 x，学习率为 0.01
optimizer = torch.optim.SGD(params=[x], lr=0.01)


# 定义函数 f(x)
def f(x):
    result = a * torch.pow(x, 2) + b * x + c
    return result


# 使用梯度下降法迭代寻找最小值
for i in range(500):
    optimizer.zero_grad()  # 清零梯度，每次迭代前都要清零
    y = f(x)  # 计算函数值
    y.backward()  # 计算梯度
    optimizer.step()  # 更新 x，使用优化器进行一步梯度下降

# 打印找到的最小值和对应的 x 值
print("最小值 y =", f(x).data.item(), "; 对应的 x =", x.data.item())
